In [ ]:
import pandas as pd
!pip install scholarly
from scholarly import scholarly

1. Extract names from .txt

In [ ]:
# Read the contents of the file
file_path = '/content/researchers.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    researcher_names = file.read().splitlines()

# Display the list of names
print(researcher_names)


['Omar Caceres', 'Ronnie G. Gavilan', 'Victor Jiménez-Vásquez, MSc.', 'Edwin Alberto Mellisho Salas', 'Michelle C. Chirinos-Arias, Ph.D.', 'Dan Erick Vivas Ruiz', 'Gustavo Adolfo Sandoval Peña', 'Pablo Sergio Ramirez Roca', 'Jorge L. Ramirez', 'Miguel Angel Chávez Fumagalli', 'Luis Tataje-Lavanda', 'MIRKO ZIMIC PERALTA', 'DIONICIA GAMBOA VILELA', 'Martha S. Calderon', 'David Requena, Ph.D.', 'Edgar Gonzalez-Kozlova', 'Yang Joon Kim', 'Pierina Danos', 'Rydberg Roman Supo Escalante', 'Ana Cecilia Romani Vasquez', 'Luis F. Soto-Ugaldi', 'Jack Aldhair Medico', 'Cesar A. Ramirez-Sarmiento', 'Phillip David Ormeño Vásquez', 'Claudio Meneses', 'mauricio arenas-salinas', 'Wendy Gonzalez', 'Ingo Dreyer', 'Estanislao Márquez', 'Miguel Angel Chávez Fumagalli', 'Arce-Johnson P & Arcejohnson & P Arce', 'David Ramirez', 'Mauricio Carrillo Tripp', 'C. Daniela Robles-Espinoza', 'Jose L. Medina-Franco', 'Alejandra Medina-Rivera', 'Javier De Las Rivas', 'Ana Conesa', 'Estefania Torrejón', 'Fernando Luis 

2. Generar el excel de los titulos de cada profile.

In [ ]:
#### First ###
# Dictionary to store paper titles by author
authors_papers = {}

# Iterate over each author, retrieve their data, and extract paper titles
for name in researcher_names:
    search_query = scholarly.search_author(name)
    try:
        author = next(search_query)
        author_data = scholarly.fill(author)
        # Extract paper titles
        titles = [pub['bib']['title'] for pub in author_data['publications']]
        # Store titles in the dictionary
        authors_papers[name] = titles
    except StopIteration:
        print(f"No information found for author: {name}")

# Create the DataFrame with researcher names and paper titles
data = {
    'Researcher': [],
    'Paper Titles': []
}

for author, titles in authors_papers.items():
    data['Researcher'].append(author)
    data['Paper Titles'].append(", ".join(titles))

df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
excel_file_path = '/content/titles_scholar_profile.xlsx'
df.to_excel(excel_file_path, index=False)

print(f"Data saved to {excel_file_path}")

2.1 Generar la data de los titulos por separado

In [ ]:
# Dictionary to store paper titles by author
authors_papers = {}

# Iterate over each author, retrieve their data, and extract paper titles
for name in researcher_names:
    search_query = scholarly.search_author(name)
    try:
        author = next(search_query)
        author_data = scholarly.fill(author)
        # Extract paper titles
        titles = [pub['bib']['title'] for pub in author_data['publications']]
        # Store titles in the dictionary
        authors_papers[name] = titles
    except StopIteration:
        print(f"No information found for author: {name}")

# Create the DataFrame with researcher names and paper titles
data = {
    'Researcher': [],
    'Paper Titles': []
}


for author, titles in authors_papers.items():
    for title in titles:
        data['Researcher'].append(author)
        data['Paper Titles'].append(title)

df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
excel_file_path = '/content/titles_scholar_profile_v2.xlsx'
df.to_excel(excel_file_path, index=False)

print(f"Data saved to {excel_file_path}")


Data saved to /content/titles_scholar_profile_v2.xlsx


3. Generar la data de los interest de los profiles

In [ ]:
def get_interest_areas(author_name):
    try:
        search_query = scholarly.search_author(author_name)
        author = next(search_query)
        author_data = scholarly.fill(author)
        return author_data.get('interests', [])
    except StopIteration:
        return []

# List to store data
researcher_data = []

for name in researcher_names:
    interest_areas = get_interest_areas(name)
    researcher_data.append({'Researcher': name, 'Interest Areas': ', '.join(interest_areas) if interest_areas else 'No interest areas found'})

# Create a DataFrame
df_interest = pd.DataFrame(researcher_data)

# Save the DataFrame to an Excel file
excel_file_path = '/content/interests_scholar_profile.xlsx'
df_interest.to_excel(excel_file_path, index=False)

print(f"Data saved to {excel_file_path}")



Data saved to /content/interests_scholar_profile.xlsx


3. Mergear ambos files

In [ ]:
import pandas as pd

# Load the data from the two Excel files
file1_path = '/content/interests_scholar_profile.xlsx'
file2_path = '/content/titles_scholar_profile.xlsx'

# Read the Excel files
df_interests = pd.read_excel(file1_path)
df_titles = pd.read_excel(file2_path)

# Merge the dataframes on the first column (assuming it's named 'Researcher' in both files)
df_merged = pd.merge(df_interests, df_titles, on='Researcher')

# Combine the second columns of each file (assuming they are named 'Interest Areas' and 'Paper Titles')
df_merged['Combined'] = df_merged['Interest Areas'] + ', ' + df_merged['Paper Titles']

# Select only the 'Researcher' and 'Combined' columns
df_output = df_merged[['Researcher', 'Combined']]

# Save the result to a new Excel file
output_file_path = '/content/papers_and_interest_sp.xlsx'
df_output.to_excel(output_file_path, index=False)

print(f"Merged data saved to {output_file_path}")

4. Formatear y traducir la data

In [ ]:
!pip install translate
from translate import Translator
import re

In [ ]:
# Initialize the translator
translator = Translator(to_lang="es")

# Unified function to translate and format terms
def translate_and_format_terms(cadena):
    try:
        # Translate the string
        translated = translator.translate(cadena)
    except Exception as e:
        print(f"Error translating '{cadena}': {e}")
        translated = cadena

    # Split terms using spaces and commas as delimiters
    terminos = re.split(r'\s*,\s*|\s+', translated)
    # Capitalize each term
    terminos = [termino.capitalize() for termino in terminos if termino]

    # Join the terms with commas
    formatted = ', '.join(terminos)
    return formatted




In [ ]:
# Load the Excel file
file_path = '/content/papers_and_interest_sp.xlsx'
df = pd.read_excel(file_path)

# Apply the translate_and_format_terms function to the second column
df['Processed'] = df['Combined'].apply(translate_and_format_terms)

# Save the processed data to a new Excel file
output_file_path = '/content/processed_papers_and_interest_sp.xlsx'
df.to_excel(output_file_path, index=False)

# Display the DataFrame
print(df)

4. Obtener los keywords en Researcher-Keywords, de cada tittle

In [ ]:
import pandas as pd

In [ ]:
# Load excel
df  = pd.read_excel('/content/titles_scholar_profile_keywords.xlsx')

# Group by 'Researcher' column and concatenate article titles
df_grouped = df.groupby('Researcher')['keywords'].apply(lambda x: ', '.join(x)).reset_index()

# Save the processed data to a new Excel file
output_file_path = '/content/keywords_scholar_profile.xlsx'
df_grouped.to_excel(output_file_path, index=False)

5. Juntar los keywords con los areas de inters en una nueva columna.

In [ ]:
# Load the excel with areas of interest
df_areas = pd.read_excel('/content/interests_scholar_profile.xlsx')

# Load the excel with keywords
df_keywords = pd.read_excel('/content/keywords_scholar_profile.xlsx')

# Merge the columns of both df based on Researcher
df_merged = pd.merge(df_areas, df_keywords, on='Researcher')

In [ ]:
#Export the df_merged into an excel
df_merged.to_excel('/content/df_merged.xlsx', index=False)

5.5 Resumir los keywords hasta máximo 50 palabras

In [ ]:
df_test_ia = df_merged.head(2)
df_test_ia

,Researcher,Interest Areas,keywords
0,Omar Caceres,"Molecular Biology, Genomics, Bioinformatics, P...","Prueba molecular, Genotype® MTBDRplus, tubercu..."
1,Ronnie G. Gavilan,"molecular epidemiology, microbial genomics, mi...","sharing, community curation, mass spectrometry..."


In [ ]:
import os
from openai import OpenAI

In [ ]:
pip install openai --upgrade


In [ ]:
import os
from openai import OpenAI

# Configura tu clave API de OpenAI (this line is probably already in a different cell)

# Pass the API key when creating the client
client = OpenAI(api_key='')  # Replace with your actual key

def get_resume_keywords(sentence):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Summarize the following list of phrases into a maximum of 40 keywords, using only the words provided, separated by commas, not periods. Exclude any information about countries, dates, institutions, or geographical regions. Focus on biological and bioinformatical topics if relevant." + sentence,
            }
        ],
        model="gpt-3.5-turbo",
    )
    keywords = chat_completion.choices[0].message.content
    return keywords


# Ejemplo de uso
#sentence = "Phylogenetic analysis, dengue virus 3, Lima-Peru, Molecular identification, genetic variants, Secuenciamiento genético, virus dengue 3, Ucayali 2004, Leishmaniosis, cutanea, clinical manifestation, H5N1, Mediane, RT-PCR."
#keywords = get_resume_keywords(sentence)
#print(keywords)

In [ ]:
df_merged

In [ ]:

from tqdm import tqdm
tqdm.pandas()
df_test_ia['resume_keywords'] = df_test_ia['keywords'].apply(get_resume_keywords)

df_test_ia


In [ ]:
from tqdm import tqdm
tqdm.pandas()
df_merged['resume_keywords'] = df_merged['keywords'].apply(get_resume_keywords)


In [ ]:
#Export df
df_merged.to_excel('/content/df_resume_keywords.xlsx', index=False)

In [ ]:
#reload the function if have more than 50 words
#len(df_merged['resume_keywords'][0].split(','))
for index, row in df_merged.iterrows():
    num_keywords = len(row['resume_keywords'].split(','))
    #show onlye keywords more than 40
    #if num_keywords > 50:
      #apply resume_keywords again only  # Volver a correr luego de la clasificación 3:46am
      df_merged['resume_keywords'] = df_merged['resume_keywords'].apply(get_resume_keywords)

    print(f"Fila {index}: {num_keywords} palabras clave")

In [ ]:
#export
df_merged.to_excel('/content/resume_keywords_v2.xlsx', index=False)

In [ ]:

df_test = df_merged

# Create a dictionary with 'Researcher' as keys and 'Interest Areas' as values
researcher_interests = df_test.set_index('Researcher')['Interest Areas'].to_dict()

# Convert to a data frame
df_researcher_interests = pd.DataFrame(list(researcher_interests.items()), columns=['Researcher', 'Interest Areas'])

# Convertir las cadenas de texto en listas
df_researcher_interests['Interest Areas'] = df_researcher_interests['Interest Areas'].apply(lambda x: x.split(', '))

# Explode the 'Interest Areas' column
df_exploded = df_researcher_interests.explode('Interest Areas')

# Reset index
df_exploded = df_exploded.reset_index(drop=True)

# Fill missing values in 'Researcher' column by forward filling
df_exploded['Researcher'] = df_exploded['Researcher'].fillna(method='ffill')

#Add data on df_test add rows
df_test_2 = pd.concat([df_test, df_exploded], ignore_index=True)

# If the cell in column 'keyword' is different of NaN, rename the cell in column 'Interest Areas' to 'Ramdom'
df_test_2.loc[df_test_2['keywords'].notna(), 'Interest Areas'] = 'Random'
df_test_2




,Researcher,Interest Areas
0,Omar Caceres,Molecular Biology
1,Omar Caceres,Genomics
2,Omar Caceres,Bioinformatics
3,Omar Caceres,Phylogeny
4,Ronnie G. Gavilan,molecular epidemiology
...,...,...
128,Cleidy Mirela Osorio Mogollon,Bioinformática
129,Cleidy Mirela Osorio Mogollon,Biofísica
130,Elizabeth Tapia,Bioinformatics
131,Elizabeth Tapia,Error Correcting Codes


In [ ]:
df_merged

In [ ]:
#Export list_of_dicts:
import json
with open('list_of_dicts.json', 'w') as f:
    json.dump(list_of_dicts, f)

In [ ]:
!pip install openai
from openai import OpenAI

import pandas as pd
import json

6. Odenar los tópicos de los keywords en base al area de interes

In [ ]:
df_test_ia = df_merged.head(2).drop('Keyword Count', axis = 1)
df_test_ia
# show number of words in cell of column 'Resume_keywords
#len(df_test_ia['resume_keywords'][2].split(','))

,Researcher,Interest Areas,keywords,resume_keywords
0,Omar Caceres,"Molecular Biology, Genomics, Bioinformatics, P...","Prueba molecular, Genotype® MTBDRplus, tubercu...","Molecular test, Genotype, Drug resistance, Evo..."
1,Ronnie G. Gavilan,"molecular epidemiology, microbial genomics, mi...","sharing, community curation, mass spectrometry...","mass spectrometry data, MS/MS networking, guid..."


In [ ]:
#Export
df_test_ia.to_excel('/content/df_test_ia.xlsx', index=False)

In [ ]:
!pip install openai
from openai import OpenAI


In [ ]:
# Antiguo
                "content":f"Categorize the following keywords based on the interest area: {interest_area}. Keywords: {keywords}. Use 'Others' as an interest area if a keyword does not have any relationship with the provided interest areas. Provide the output as a JSON list of dictionaries, where each dictionary contains a 'keyword' and its 'interest_area'. Ensure the output is in a consistent format, like the following example:\n[\n\
                ```json\n\
                [\n\
                    {{'keyword': 'Phylogeny', 'interest_area': 'Molecular Systematics'}},\n\
                     {{'keyword': 'species delimitations', 'interest_area': 'Molecular Systematics'}},\n\
                      {{'keyword': 'genetic diversity', 'interest_area': 'Genomics'}}\n\
                ]\n\
                ``` , Also If you detect spelling errors, please correct them and Translate all keywords in English"
                #f"Categorize the following keywords based on the interest area: {interest_area}. Keywords: {keywords}. Use 'Others' as an interest area if does't have any relationship with the interest areas \n Provide the output as a list of dictionaries, where each dictionary contains a 'keyword' and its 'interest_area'."
                #"content": f"Categorize the following keywords based on the interest area: {interest_area}. Keywords: {keywords}. \n Put all the classified keywords in a list, separated with commas."


In [6]:
client = OpenAI(api_key='')  # Replace with your actual key

def categorize_keywords(keywords, interest_area):
     chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant programmer expert on Bioinformatical and Biological topics that categorizes keywords based on given interest areas."
            },
            {
                "role": "user",
                "content": (
                    f"Categorize the following keywords based on the interest area: {interest_area}. "
                    f"Keywords: {keywords}. Use 'Others' as an interest area if a keyword does not have any relationship with the provided interest areas. "
                    "Provide the output as a JSON list of dictionaries, where each dictionary contains a 'keyword' and its 'interest_area'. "
                    "Ensure the output is in a consistent format, like the following example:\n"
                    "[\n"
                    "    ```json\n"
                    "    [\n"
                    "        {'keyword': 'Phylogeny', 'interest_area': 'Molecular Systematics'},\n"
                    "        {'keyword': 'species delimitations', 'interest_area': 'Molecular Systematics'},\n"
                    "        {'keyword': 'genetic diversity', 'interest_area': 'Genomics'}\n"
                    "    ]\n"
                    "    ```\n"
                    "]\n"
                    "Also If you detect spelling errors, please correct them. And Translate all keywords in English"
                )
},
        ], model="gpt-3.5-turbo",
    )
     return chat_completion.choices[0].message.content

# Ejemplo de uso
keywords = " Banana, Prueba molecular, Genotype® MTBDRplus, tuberculosis multidrogorresistente, Evolutionary, genomic dynamics"
interest_area = "Genomics and Molecular Biology", "Biotechnology", 'Medicine'
categorized_keywords = categorize_keywords(keywords, interest_area)
print(categorized_keywords)


```json
[
    {'keyword': 'Banana', 'interest_area': 'Others'},
    {'keyword': 'Molecular test', 'interest_area': 'Medicine'},
    {'keyword': 'Genotype® MTBDRplus', 'interest_area': 'Medicine'},
    {'keyword': 'Drug-resistant tuberculosis', 'interest_area': 'Medicine'},
    {'keyword': 'Evolutionary', 'interest_area': 'Others'},
    {'keyword': 'Genomic dynamics', 'interest_area': 'Genomics and Molecular Biology'}
]
```


Con la data de prueba

In [ ]:
#import
df_resume_keywords = pd.read_excel('/content/resume_keywords_v2.xlsx')

from tqdm import tqdm
tqdm.pandas()
#Use the function
df_resume_keywords['Classified Keywords'] = df_resume_keywords.progress_apply(lambda row: categorize_keywords(row['resume_keywords'], row['Interest Areas']), axis=1)

#export
df_resume_keywords.to_excel('/content/classified_keywords_v3.xlsx', index=False)

100%|██████████| 43/43 [05:40<00:00,  7.92s/it]


In [ ]:
# Use columns Researcher, Interest Areas, Classified Keywords from df_test_ia
df_test_ia_necesary_columns = df_test_ia[['Researcher', 'Interest Areas', 'Classified Keywords']]

# Export
df_test_ia_necesary_columns.to_excel('/content/df_test_ia_necesary_columns.xlsx', index=False)

df_exploded = df_test_ia_necesary_columns.explode('Classified Keywords')
df_exploded

,Researcher,Interest Areas,Classified Keywords
0,Omar Caceres,"Molecular Biology, Genomics, Bioinformatics, P...","```json\n[\n {""keyword"": ""Molecular test"", ..."
1,Ronnie G. Gavilan,"molecular epidemiology, microbial genomics, mi...","[\n {""keyword"": ""mass spectrometry data"", ""..."


Con la data original

In [ ]:
tqdm.pandas()
#ust the function to df_merged
df_merged['Classified Keywords'] = df_merged.progress_apply(lambda row: categorize_keywords(row['resume_keywords'], row['Interest Areas']), axis=1)

#export
df_merged.to_excel('/content/interest_and_classified_keywords_v2.xlsx', index=False)

100%|██████████| 43/43 [05:26<00:00,  7.58s/it]


Procesar test de keywords clasificados

In [ ]:
# Función para convertir el JSON string a lista de diccionarios
### Función antigue
def parse_classified_keywords(json_str):
  if '```json' in json_str:
    json_str = json_str.replace('```json', '').replace('```', '').strip()  # Limpiar el string
    return json.loads(json_str)
  else:
    json_str = json_str.strip()
    return json.loads(json_str)


In [ ]:
type(df.loc[2,'Classified Keywords'])

str

In [ ]:
df_merged

In [ ]:
# import df_merged
import pandas as pd
df_merged = pd.read_excel('/content/interest_and_classified_keywords_v2.xlsx')

#Use only 3 columns
df_merged = df_merged[['Researcher', 'Interest Areas', 'Classified Keywords']]
#df_merged

In [ ]:
import json
import ast
import pandas as pd

def parse_classified_keywords(json_str):
    # Imprimir el valor original para depuración
    print(f"Original: {json_str}")

    # Eliminar los caracteres de nueva línea
    json_str = json_str.replace("\n", "").strip()

    # Verificar si el string contiene '```json' y limpiarlo
    if '```json' in json_str:
        json_str = json_str.replace('```json', '').replace('```', '').strip()

    # Intentar cargar el string como JSON
    try:
        result = json.loads(json_str)
        print(f"JSON válido: {result}")
        return result
    except json.JSONDecodeError as e:
        print(f"Error de JSON: {e}")
        # Si falla, intentar procesarlo como un string de diccionarios con comillas simples
        try:
            json_str = json_str.replace("'", '"')
            result = json.loads(json_str)
            print(f"Comillas simples convertidas a dobles: {result}")
            return result
        except json.JSONDecodeError as e:
            print(f"Error de JSON después de convertir comillas: {e}")
            # Si aún falla, intentar usar ast.literal_eval
            try:
                result = ast.literal_eval(json_str)
                print(f"Evaluado con ast.literal_eval: {result}")
                return result
            except (ValueError, SyntaxError) as e:
                print(f"Error de evaluación literal: {e}")
                raise ValueError("El string proporcionado no es un JSON válido ni una lista de diccionarios.")

# Reemplazar \n en la columna 'Classified Keywords'
df_merged.loc[:, 'Classified Keywords'] = df_merged['Classified Keywords'].apply(lambda x: x.replace('\n', ''))

# Aplicar la función a la columna 'Classified Keywords'
try:
    df_merged['new_Classified Keywords'] = df_merged['Classified Keywords'].apply(parse_classified_keywords)
except ValueError as e:
    print(f"Error al procesar la columna: {e}")

df_merged


In [ ]:
import pandas as pd
import json

df = df_merged.head(3)
df.loc[:, 'Classified Keywords'] = df['Classified Keywords'].apply(lambda x: x.replace('\n', ''))

# Replace \n in column Classified keywords
df['Classified Keywords'] = df['Classified Keywords'].apply(lambda x: x.replace('\n', ''))

df

# Aplicar la función a la columna 'Classified Keywords'
df['new_Classified Keywords'] = df['Classified Keywords'].apply(parse_classified_keywords)

df

#export
#df.to_excel('/content/df_test.xlsx', index=False)

# Usar explode para expandir las listas de diccionarios en filas separadas
df_exploded = df.explode('new_Classified Keywords')
df_exploded


# Convertir cada diccionario en columnas separadas
df_exploded = pd.concat([df_exploded.drop(['new_Classified Keywords'], axis=1), df_exploded['new_Classified Keywords'].apply(pd.Series)], axis=1)
df_exploded.drop(['Interest Areas'], axis = 1)
df_exploded
#Export
df_exploded.to_excel('/content/df_exploded_test.xlsx', index=False)
# Renombrar las columnas según sea necesario
df_exploded = df_exploded.rename(columns={'keyword': 'Keywords', 'interest_area': 'Interest Area'})

# Cambiar el orden de las columnas
df_exploded = df_exploded[['Researcher', 'Interest Area', 'Keywords']]

# Mostrar el DataFrame final
print(df_exploded)

# Guardar el DataFrame final en un nuevo archivo Excel
output_file_path = '/mnt/data/df_test_ia_exploded.xlsx'
df_exploded.to_excel(output_file_path, index=False)
output_file_path


,Researcher,Interest Areas,Classified Keywords
0,Omar Caceres,"Molecular Biology, Genomics, Bioinformatics, P...","```json[ {""keyword"": ""Molecular test"", ""int..."
1,Ronnie G. Gavilan,"molecular epidemiology, microbial genomics, mi...","```json[ {""keyword"": ""mass spectrometry dat..."
2,"Victor Jiménez-Vásquez, MSc.","Molecular Epidemiology, Molecular Biology, Evo...","[ {'keyword': 'Genomic analysis', 'interest..."


In [ ]:
# Confirmar si está bien la cantidad de rows.
df_test_ia
for index, row in df_test_ia.iterrows():
    num_keywords = len(row['resume_keywords'].split(','))
    print(f"Fila {index}: {num_keywords} palabras clave")

,Researcher,Interest Areas,keywords,resume_keywords,Classified Keywords
0,Omar Caceres,"Molecular Biology, Genomics, Bioinformatics, P...","Prueba molecular, Genotype® MTBDRplus, tubercu...","Molecular test, Genotype, Drug resistance, Evo...","```json\n[\n {""keyword"": ""Molecular test"", ..."
1,Ronnie G. Gavilan,"molecular epidemiology, microbial genomics, mi...","sharing, community curation, mass spectrometry...","mass spectrometry data, MS/MS networking, guid...","[\n {""keyword"": ""mass spectrometry data"", ""..."


Procesar todos los keywords clasificados

In [ ]:
# Use columns Researcher, Interest Areas, Classified Keywords from df_test_ia
df_merged_3_column = df_merged[['Researcher', 'Interest Areas', 'Classified Keywords']]


In [ ]:

# Función para convertir el JSON string a lista de diccionarios
def parse_classified_keywords(json_str):
    json_str = json_str.replace('```json', '').replace('```', '').strip()  # Limpiar el string
    return json.loads(json_str)


In [ ]:
# Función para convertir el JSON string a lista de diccionarios
def parse_classified_keywords(json_str):
    try:
        # Limpiar el string
        json_str = json_str.replace('```json', '').replace('```', '').strip()
        # Reemplazar comillas simples por comillas dobles
        json_str = re.sub(r"'", r'"', json_str)
        # Asegurarse de que las claves del diccionario estén entre comillas dobles
        json_str = re.sub(r'(\w+):', r'"\1":', json_str)
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return []

In [ ]:
import pandas as pd
import json
import re

df = df_merged_3_column

# Aplicar la función a la columna 'Classified Keywords'
df['Classified Keywords'] = df['Classified Keywords'].apply(parse_classified_keywords)

# Usar explode para expandir las listas de diccionarios en filas separadas
df_exploded = df.explode('Classified Keywords')

# Convertir cada diccionario en columnas separadas
df_exploded = pd.concat([df_exploded.drop(['Classified Keywords'], axis=1), df_exploded['Classified Keywords'].apply(pd.Series)], axis=1)
df_exploded.drop(['Interest Areas'], axis = 1)

# Renombrar las columnas según sea necesario
df_exploded = df_exploded.rename(columns={'keyword': 'Keywords', 'interest_area': 'Interest Area'})

# Cambiar el orden de las columnas
df_exploded = df_exploded[['Researcher', 'Interest Area', 'Keywords']]

# Mostrar el DataFrame final
print(df_exploded)

# Guardar el DataFrame final en un nuevo archivo Excel
output_file_path = '/content/processed_interest_and_classified_keywords_v1.xlsx'
df_exploded.to_excel(output_file_path, index=False)


Categorizar los keywords crudos

In [8]:
#import
import pandas as pd
df_keywords = pd.read_excel('/content/interests_and_keywords_scholar_profile.xlsx')

In [10]:
from tqdm import tqdm
tqdm.pandas()
#Use the function
df_keywords['Classified Raw Keywords'] = df_keywords.progress_apply(lambda row: categorize_keywords(row['keywords'], row['Interest Areas']), axis=1)



100%|██████████| 43/43 [14:45<00:00, 20.60s/it]


In [11]:
#Export df_keywords
df_keywords.to_excel('/content/classified_raw_keywords_v1.xlsx', index=False)

In [ ]:
import pandas as pd
import json
import re

df = df_merged_3_column

# Aplicar la función a la columna 'Classified Keywords'
df['Classified Keywords'] = df['Classified Keywords'].apply(parse_classified_keywords)

# Usar explode para expandir las listas de diccionarios en filas separadas
df_exploded = df.explode('Classified Keywords')

# Convertir cada diccionario en columnas separadas
df_exploded = pd.concat([df_exploded.drop(['Classified Keywords'], axis=1), df_exploded['Classified Keywords'].apply(pd.Series)], axis=1)
df_exploded.drop(['Interest Areas'], axis = 1)

# Renombrar las columnas según sea necesario
df_exploded = df_exploded.rename(columns={'keyword': 'Keywords', 'interest_area': 'Interest Area'})

# Cambiar el orden de las columnas
df_exploded = df_exploded[['Researcher', 'Interest Area', 'Keywords']]

# Mostrar el DataFrame final
print(df_exploded)

# Guardar el DataFrame final en un nuevo archivo Excel
output_file_path = '/content/processed_interest_and_classified_keywords_v1.xlsx'
df_exploded.to_excel(output_file_path, index=False)